In [22]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import os
from llama_index.core.prompts import PromptTemplate
from lecture_formatting import lecture_dates6100L, get_status, create_documents, lecture_mapping
from datetime import datetime
from pydantic import BaseModel
from enum import Enum
from typing_extensions import Literal


In [36]:
class IsSeen(BaseModel):
    """An object determining if a document has been seen or not"""
    seen: bool
class IsSup(BaseModel):
    """An object determining if a document fully support, partially supprt or none support for an answer"""
    sup: Literal["fully", "partial", "none"]
    
class IsUse(BaseModel):
    """An object determining how useful an answer is from 1 to 5"""
    use: Literal["1", "2", "3", "4", "5"]

class IsRel(BaseModel):
    """An object determine if the retrieved passage is relevant to the question"""
    rel: bool

class IsRet(BaseModel):
    """An object determining whether a question requires retrieval of external documents"""
    ret: bool

In [5]:
load_dotenv()
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [6]:
def get_llm(model):
    """
    Function that return LLMs given model and extra arguments
    TODO: Add more LLMs
    """
    return OpenAI(model=model)

In [37]:
def get_score(seen_obj, rel_obj, sup_obj, use_obj):
    w = [0.2, 0.3, 0.2, 0.4]
    seen = 1 if seen_obj.seen else 0
    rel = 1 if rel_obj.rel else 0
    sup = 0
    if sup_obj.sup == "fully":
        sup = 1
    elif sup_obj.sup == "partial":
        sup = 0.5
    use = int(use_obj.use)
    return w[0]*seen + w[1]*rel + w[2]*sup + w[3]*use 


In [10]:
embed_model=LangchainEmbedding(HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1"))
llm = get_llm("gpt-4")

service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

/Users/gyalpoaguirre/Dropbox/Mac/Documents/mit/6.s893/self-rag-education/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/gyalpoaguirre/Dropbox/Mac/Documents/mit/6.s893/self-rag-education/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/var/folders/q_/gmxdkf893w3bm9wxvh6635t80000gp/T/ipykernel_53460/2349502471.py:4: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [25]:
documents=SimpleDirectoryReader("./formatted_lectures").load_data()
index=VectorStoreIndex.from_documents(documents,service_context=service_context)


In [30]:
retriever = index.as_retriever(similarity_top_k=5)

In [42]:
def get_response(question, student_date):
    response = ""
    prompt = "You are a teaching assistant helping a student answer different questions."
    retrieval_tmpl = PromptTemplate("Generate a Retrieval object given this question {question}")
    ret_obj = llm.structured_predict(IsRet, retrieval_tmpl, question=question)
    if ret_obj.ret:
        retrieved_nodes = retriever.retrieve(question) 
        seen_tmpl = PromptTemplate("Generate an IsSeen object given the fact that the student has seen all lectures up to this date {student_date} and this current lecture has been given on this date {lecture_date}.")
        rel_tmpl = PromptTemplate("Generate an IsRel object given a passage:{passage} and question: {question} toetermine if this passage is relevant to this question")
        use_tmpl  = PromptTemplate("Generate an IsUse object by ranking how useful the response: {response} for this question: {question} is from 1 to 5")
        sup_tmpl = PromptTemplate("Generate an IsSup object by evaluating how supportive the response: {response} for this question:{question}, whether it is 'fully', 'partial' or 'none' support.")
        nodes = []
        for node in retrieved_nodes:
            lecture = node.node.metadata["file_name"].split(".md")[0]
            lecture_date = lecture_dates6100L[lecture]
            seen_obj = llm.structured_predict(IsSeen, seen_tmpl, student_date=student_date, lecture_date=lecture_date)
            passage = node.node.text
            rel_obj = llm.structured_predict(IsRel, rel_tmpl, passage=passage, question=question)
            temp_prompt = prompt + f"\nThe following is an excerpt which could be useful for answering this question: {passage}"
            messages = [
                ChatMessage(role="system", content=temp_prompt),
                ChatMessage(role="user", content=question),
            ]
            temp_response =  llm.chat(messages).message.content
            use_obj = llm.structured_predict(IsUse, use_tmpl, response=temp_response, question=question)
            sup_obj = llm.structured_predict(IsSup, sup_tmpl, response=temp_response, question=question)
            rag_score = get_score(seen_obj, rel_obj, sup_obj, use_obj)
            nodes.append((rag_score, node))
        nodes.sort(key=lambda x: x[0])
        filtered_nodes = [node for (rag_score,node) in nodes if rag_score > 0.5][:3]
        if filtered_nodes:
            prompt += "\nThe following are passages from the lecture notes which can (but not necessary to) help you answer the question:\n"
            for node in filtered_nodes:
                prompt += f"{node.node.text}\n"
    messages = [
            ChatMessage(role="system", content=prompt),
            ChatMessage(role="user", content=question),
        ]
    response = llm.chat(messages).message.content
    return response

In [43]:
response = get_response("What is a list in python?", "10/24/2022")


In [44]:
response

"A list in Python is a built-in data structure that can hold a collection of items. These items can be of different types including integers, strings, other lists, dictionaries, etc. Lists are mutable, meaning their elements can be changed after they are created. They are ordered, which means that the items have a defined order that will not change unless you do so explicitly. Lists are defined by having values between square brackets [ ] and items are separated by commas. For example, a list could be: [1, 'a', [2, 3], 'hello']."